In [8]:
#!/usr/bin/env python
# coding: utf-8

# # EfficientDet을 이용한 과일 품질 탐지 프로젝트
#
# 이 스크립트는 `efficientdet-pytorch` 저장소를 활용하여 과일(사과, 배, 감)의 품질(특상, 상, 중)을 탐지하는 모델을 학습하기 위한 데이터 준비 및 설정 파일 생성을 수행합니다.
#
# **전제 조건:**
# 1.  `effdet_env` conda 가상 환경에 `torch`, `pandas`, `numpy`, `pycocotools`, `opencv-python`, `PyYAML` 등이 설치되어 있어야 합니다.
# 2.  `efficientdet-pytorch` 저장소는 `zylo117/Yet-Another-EfficientDet-Pytorch`를 기준으로 합니다.
# 3.  `fruits/datasets/json_labels` 내의 JSON 파일에 바운딩 박스 좌표가 포함되어 있어야 합니다.

import os
import sys
import glob
import json
import pandas as pd
import numpy as np
import yaml
import shutil

In [9]:


# ## 1. 경로 설정
#
# 스크린샷의 디렉토리 구조를 기반으로 경로를 설정합니다.

print("--- 1. 경로 설정 시작 ---")
# 현재 스크립트 파일의 위치를 기준으로 상대 경로 설정
# (fruits/efficientdet/ 이므로 두 단계 위로 올라가야 함)
# [수정] __file__는 .py 스크립트 실행 시에만 존재합니다.
# Jupyter Notebook(.ipynb)에서는 os.getcwd()를 사용해야 합니다.
# CURRENT_FILE_DIR = os.path.dirname(os.path.abspath(__file__)) # 원본 .py 파일용
CURRENT_FILE_DIR = os.getcwd() # Jupyter Notebook 호환용
ROOT_DIR = os.path.abspath(os.path.join(CURRENT_FILE_DIR, '../../'))

DATA_DIR = os.path.join(ROOT_DIR, 'data/row')
IMAGE_DIR = os.path.join(DATA_DIR, 'images')
JSON_DIR = os.path.join(DATA_DIR, 'json_labels')

EFFDET_REPO_DIR = os.path.join(ROOT_DIR, 'model/efficientdet-pytorch')

# efficientdet-pytorch 저장소의 모듈을 임포트할 수 있도록 경로 추가
if EFFDET_REPO_DIR not in sys.path:
    sys.path.append(EFFDET_REPO_DIR)

print(f"Root Directory: {ROOT_DIR}")
print(f"Data Directory: {DATA_DIR}")
print(f"Image Directory: {IMAGE_DIR}")
print(f"JSON Directory: {JSON_DIR}")
print(f"EfficientDet Repo: {EFFDET_REPO_DIR}")

# 학습된 모델과 설정 파일이 저장될 위치
OUTPUT_DIR = os.path.join(CURRENT_FILE_DIR, 'outputs')
os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"Output Directory: {OUTPUT_DIR}")
print("--- 1. 경로 설정 완료 ---\n")







--- 1. 경로 설정 시작 ---
Root Directory: c:\Github\class\z_fruit_quality_project
Data Directory: c:\Github\class\z_fruit_quality_project\data/row
Image Directory: c:\Github\class\z_fruit_quality_project\data/row\images
JSON Directory: c:\Github\class\z_fruit_quality_project\data/row\json_labels
EfficientDet Repo: c:\Github\class\z_fruit_quality_project\model/efficientdet-pytorch
Output Directory: c:\Github\class\z_fruit_quality_project\src\efficientdet\outputs
--- 1. 경로 설정 완료 ---



In [10]:


# ## 2. 데이터 전처리
#
# `json_labels`의 JSON 파일들을 `efficientdet-pytorch`가 요구하는 학습용 `train.csv`와 `val.csv` 파일로 변환합니다.
#
# **JSON 구조 확인 (샘플 기준):**
# JSON 파일 하나가 하나의 이미지, 하나의 객체(과일)에 대한 정보를 담고 있습니다.
# ```json
# {
#     "identifier": "53.png",
#     "cate1": "사과",
#     "cate3": "특",
#     "format": "png",
#     "bndbox": {
#         "xmin": 0,
#         "ymin": 0,
#         "xmax": 1000,
#         "ymax": 1000
#     }
# }
# ```
# 이 구조를 바탕으로 `image_path,x1,y1,x2,y2,class_name` 형식의 CSV 파일을 생성합니다.

def parse_json_file(json_path, image_dir):
    """
    제공된 JSON 샘플 구조에 맞춰 파싱 함수를 수정합니다.
    JSON 파일 하나당 하나의 어노테이션을 처리합니다.
    """
    annotations = []
    all_labels = set()
    
    try:
        with open(json_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        # 1. 이미지 파일명 가져오기
        # [대폭 수정] 모든 복잡한 파일명 파싱 로직을 (if/elif/else) 제거합니다.
        # AI Hub 데이터셋의 표준인 'repo'와 'identifier' 필드를 직접 사용합니다.
        
        image_name = data.get('identifier') # 예: "83.png" 또는 "57.png"
        image_repo = data.get('repo')       # 예: "apple/fuji/S/26-57" 또는 "사과/부사/상/26"

        if image_repo and image_name:
            # [추가] repo 경로에 Windows 스타일 '\'가 포함된 경우, '/'로 변경
            normalized_repo = image_repo.replace('\\', os.path.sep)
            
            # repo 필드가 있으면, 하위 폴더 경로를 포함하여 image_path 구성
            image_path = os.path.join(image_dir, normalized_repo, image_name)
        elif image_name:
            # repo 필드가 없는 경우 (대비용)
            image_path = os.path.join(image_dir, image_name)
        else:
            print(f"Warning: JSON file {json_path} is missing 'identifier' or 'repo'.")
            image_path = None # 찾기 실패

        if not image_path or not os.path.exists(image_path):
            print(f"Warning: Image file not found {image_path} (from {json_path})")
            return [], set()

        # 2. 레이블 및 바운딩 박스 정보 가져오기
        cate1 = data.get('cate1')
        cate3 = data.get('cate3')
        bndbox = data.get('bndbox')
        
        if bndbox and cate1 and cate3:
            x1 = bndbox.get('xmin')
            y1 = bndbox.get('ymin')
            x2 = bndbox.get('xmax')
            y2 = bndbox.get('ymax')
            
            # 모든 좌표값이 유효한지 확인
            if all(isinstance(v, (int, float)) for v in [x1, y1, x2, y2]):
                # 클래스 레이블 조합: "사과-특"
                class_name = f"{cate1}-{cate3}"
                all_labels.add(class_name)
                
                # CSV 형식: image_path,x1,y1,x2,y2,class_name
                annotations.append([image_path, x1, y1, x2, y2, class_name])
            else:
                print(f"Warning: Incomplete bbox coordinates in {json_path}")
        else:
            print(f"Warning: Missing 'cate1', 'cate3', or 'bndbox' in {json_path}")
                
    except Exception as e:
        print(f"Error parsing {json_path}: {e}")
        return [], set()

    return annotations, all_labels

def create_annotation_csv(json_dir, image_dir, output_path):
    json_files = glob.glob(os.path.join(json_dir, "*.json"))
    
    all_annotations = []
    all_class_names = set()
    
    print(f"Found {len(json_files)} JSON files.")

    for json_path in json_files:
        annotations, class_names = parse_json_file(json_path, image_dir)
        all_annotations.extend(annotations)
        all_class_names.update(class_names)

    if not all_annotations:
        print("Error: No annotations were processed. Check JSON files or image paths.")
        return None
    
    # DataFrame 생성
    df = pd.DataFrame(all_annotations, columns=['image_path', 'x1', 'y1', 'x2', 'y2', 'class_name'])
    
    # CSV 파일로 저장 (efficientdet-pytorch는 헤더가 없는 CSV를 사용)
    df.to_csv(output_path, index=False, header=False)
    
    print(f"Annotation file created: {output_path} with {len(df)} entries.")
    
    sorted_classes = sorted(list(all_class_names))
    print(f"Total unique classes found ({len(sorted_classes)}): {sorted_classes}")
    
    return sorted_classes


# --- 전처리 실행 ---
print("--- 2. 데이터 전처리 시작 ---")

# [참고] 여기서는 모든 데이터를 train.csv로 만듭니다. 
# 실제 프로젝트에서는 train/validation set을 나눠야 합니다. (예: 80% train, 20% val)
# (예: glob으로 파일 리스트를 가져온 후, sklearn의 train_test_split으로 분리)
# 지금은 편의상 train/val에 동일한 CSV를 사용합니다.

TRAIN_CSV_PATH = os.path.join(OUTPUT_DIR, 'train.csv')
VAL_CSV_PATH = os.path.join(OUTPUT_DIR, 'val.csv')

class_list = create_annotation_csv(JSON_DIR, IMAGE_DIR, TRAIN_CSV_PATH)

if class_list:
    # Validation set을 따로 나누지 않았으므로, train.csv를 그대로 복사
    shutil.copy(TRAIN_CSV_PATH, VAL_CSV_PATH)
    print(f"Validation CSV created (copied from train.csv): {VAL_CSV_PATH}")
    print("--- 2. 데이터 전처리 완료 ---\n")
else:
    print("!!! FAILED to create annotation files. Please check console warnings. !!!")
    print("--- 2. 데이터 전처리 실패 ---\n")


--- 2. 데이터 전처리 시작 ---
Found 70 JSON files.
Error: No annotations were processed. Check JSON files or image paths.
!!! FAILED to create annotation files. Please check console warnings. !!!
--- 2. 데이터 전처리 실패 ---



In [11]:
# ## 3. 프로젝트 YAML 설정 파일 생성
#
# `efficientdet-pytorch` (`zylo117`) 저장소는 학습을 위해 `.yml` 설정 파일을 사용합니다.

print("--- 3. YAML 설정 파일 생성 시작 ---")
if class_list:
    project_name = 'fruits_quality_detection'
    num_classes = len(class_list)
    
    project_config = {
        'project_name': project_name,
        'train_set': TRAIN_CSV_PATH,
        'val_set': VAL_CSV_PATH,
        'num_gpus': 1,  # [TODO] 사용 가능한 GPU 수에 맞게 수정 (0이면 CPU)
        
        # ImageNet 기본 값. 데이터셋에 맞게 조절 가능
        'mean': [0.485, 0.456, 0.406],
        'std': [0.229, 0.224, 0.225],
        
        # 클래스 리스트 (obj_list)
        'obj_list': class_list
    }

    PROJECT_YML_PATH = os.path.join(OUTPUT_DIR, f"{project_name}.yml")
    
    with open(PROJECT_YML_PATH, 'w', encoding='utf-8') as f:
        yaml.dump(project_config, f, allow_unicode=True, default_flow_style=False)
        
    print(f"Project config file created: {PROJECT_YML_PATH}")
    print("--- Config File Content ---")
    with open(PROJECT_YML_PATH, 'r') as f:
        print(f.read())
    print("--- 3. YAML 설정 파일 생성 완료 ---\n")
else:
    print("Class list is empty. Skipping YAML creation.")
    print("--- 3. YAML 설정 파일 생성 실패 ---\n")




--- 3. YAML 설정 파일 생성 시작 ---
Class list is empty. Skipping YAML creation.
--- 3. YAML 설정 파일 생성 실패 ---



In [12]:
# ## 4. 모델 학습 실행
#
# 터미널에서 `efficientdet-pytorch` 폴더의 `train.py`를 실행합니다.
#
# `-c 0`: 사용할 EfficientDet 모델 번호 (0~7). 0은 D0 (가장 작음) 입니다.
# `--batch_size`: GPU 메모리에 맞게 조절합니다.
# `--lr`: Learning rate
# `--num_epochs`: 학습 에포크 수
# `-p`: 방금 생성한 프로젝트 `.yml` 파일 경로
# `--weights`: 사전 학습된 가중치 경로 (없으면 `None` 또는 `coco` 가중치 사용)
#
# **참고:** 이 스크립트는 학습 명령어를 출력만 해줍니다. 실제 학습은 터미널에서 실행해야 합니다.

print("--- 4. 모델 학습 명령어 생성 ---")
# [TODO] 파라미터를 수정하세요.
# pretrained_weights.pth는 COCO 등으로 사전 학습된 가중치 파일입니다. 
# 저장소에서 다운로드 받거나, None으로 두어 처음부터 학습할 수 있습니다.
# (일반적으로 COCO 가중치에서 시작하는 것이 좋습니다)

# 예: COCO 사전학습 가중치 사용 (D0 모델)
# !wget https://github.com/zylo117/Yet-Another-EfficientDet-Pytorch/releases/download/1.0/efficientdet-d0.pth -P {EFFDET_REPO_DIR}
# PRETRAINED_WEIGHTS = os.path.join(EFFDET_REPO_DIR, 'efficientdet-d0.pth')

# 가중치 없이 시작
PRETRAINED_WEIGHTS = 'None'

if class_list:
    train_command = f"python {EFFDET_REPO_DIR}/train.py \\\n" \
                     f"                     -c 0 \\\n" \
                     f"                     --batch_size 4 \\\n" \
                     f"                     --lr 1e-3 \\\n" \
                     f"                     --num_epochs 50 \\\n" \
                     f"                     -p {PROJECT_YML_PATH} \\\n" \
                     f"                     --weights {PRETRAINED_WEIGHTS} \\\n" \
                     f"                     --val_interval 1 \\\n" \
                     f"                     --save_interval 5 \\\n" \
                     f"                     --log_path {OUTPUT_DIR}/logs"
    
    print("--- Training Command ---")
    print("# 아래 명령어를 터미널에서 실행하세요 (conda 'effdet_env' 활성화 필수):")
    print(f"# cd {EFFDET_REPO_DIR}")
    print(f"# conda activate effdet_env")
    print(train_command.replace(f"python {EFFDET_REPO_DIR}/", "python "))
    print("--- 4. 모델 학습 명령어 생성 완료 ---\n")
else:
    print("학습 파일을 생성하지 못해 명령어를 만들 수 없습니다.")
    print("--- 4. 모델 학습 명령어 생성 실패 ---\n")




--- 4. 모델 학습 명령어 생성 ---
학습 파일을 생성하지 못해 명령어를 만들 수 없습니다.
--- 4. 모델 학습 명령어 생성 실패 ---



In [13]:
# ## 5. 추론 (Inference)
#
# 학습이 완료된 후, `validate.py` (또는 별도 스크립트)를 사용하여 새로운 이미지로 테스트합니다.

print("--- 5. 추론 명령어 생성 ---")
if class_list:
    # [TODO] 학습이 완료된 후 생성된 가중치 파일 경로 (.pth)
    # 예: {OUTPUT_DIR}/logs/fruits_quality_detection/d0_50.pth
    TRAINED_MODEL_PATH = os.path.join(OUTPUT_DIR, 'logs', project_name, 'd0_50.pth') # 50 에포크 완료 시 예시
    
    # [TODO] 테스트할 이미지 경로
    TEST_IMAGE_PATH = "path/to/your/test_image.jpg"

    inference_command = f"python {EFFDET_REPO_DIR}/validate.py \\\n" \
                         f"                         -p {PROJECT_YML_PATH} \\\n" \
                         f"                         -w {TRAINED_MODEL_PATH} \\\n" \
                         f"                         --img_path {TEST_IMAGE_PATH} \\\n" \
                         f"                         --visualize"
    
    print("--- Inference Command (Example) ---")
    print("# 아래 명령어를 실행하여 테스트 이미지에 대한 탐지 결과를 확인합니다.")
    print("# (TRAINED_MODEL_PATH와 TEST_IMAGE_PATH를 실제 경로로 수정해야 합니다)")
    print(inference_command)
    print("\n# (만약 validate.py가 시각화 옵션을 지원하지 않는다면, 별도 inference.py를 작성해야 할 수 있습니다)")
    print("--- 5. 추론 명령어 생성 완료 ---\n")

print("--- 스크립트 실행 완료 ---")

--- 5. 추론 명령어 생성 ---
--- 스크립트 실행 완료 ---
